In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
DATASET = 'data/'
TRAIN_CSV = 'cs-training.csv'
TEST_CSV = 'cs-test.csv'

def load_train():
    path = DATASET+TRAIN_CSV
    return pd.read_csv(path)

def load_test():
    path = DATASET+TEST_CSV
    return pd.read_csv(path)


train= load_train()


train['NumberOfTime30-59DaysPastDueNotWorse'].loc[(train['NumberOfTime30-59DaysPastDueNotWorse']==98) | (train['NumberOfTime30-59DaysPastDueNotWorse']==96)] = train['NumberOfTime30-59DaysPastDueNotWorse'].median()



train =train.drop('#',axis=1)


c:\users\dell\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\dell\appdata\local\programs\python\python35\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
c:\users\dell\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [2]:
train['MonthlyDebts'] = train['DebtRatio']*train['MonthlyIncome']
train =train.drop('DebtRatio',axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null int64
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null int64
NumberRealEstateLoansOrLines            150000 non-null int64
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null int64
NumberOfDependents                      146076 non-null float64
MonthlyDebts                            120269 non-null float64
dtypes: float64(5), int64(6)
memory usage: 12.6 MB


In [3]:
from sklearn.preprocessing import Imputer
#imputing data


train['NumberOfDependents'] = train['NumberOfDependents'].fillna(value=0)
imputer = Imputer(strategy='median')
imputer.fit(train)
x = imputer.transform(train)
X = pd.DataFrame(x,columns=train.columns)
X_ = X.drop('SeriousDlqin2yrs',axis = 1)
Y = train['SeriousDlqin2yrs']






c:\users\dell\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_,Y)



print(round(model.score(X_,Y)*100,2))
clf = cross_validate(model,X_,Y)
print(round(clf['train_score'].mean()*100,2))
print(round(clf['test_score'].mean()*100,2))

   

99.88


c:\users\dell\appdata\local\programs\python\python35\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


99.9
93.51


c:\users\dell\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [5]:
test = load_test()
test = test.drop('#',axis=1)
Y1 = test['SeriousDlqin2yrs']
X1 = test.drop('SeriousDlqin2yrs',axis=1)


X1['NumberOfDependents'].fillna(value=0)
test['NumberOfTime30-59DaysPastDueNotWorse'].loc[(test['NumberOfTime30-59DaysPastDueNotWorse']==98) | (test['NumberOfTime30-59DaysPastDueNotWorse']==96)] = test['NumberOfTime30-59DaysPastDueNotWorse'].median()
imputer.fit(X1)
x_ = imputer.transform(X1)
x1 = pd.DataFrame(x_,columns=X1.columns)



x1['MonthlyDebts'] = x1['DebtRatio']*x1['MonthlyIncome']
x1 =x1.drop('DebtRatio',axis=1)
x1.info()


c:\users\dell\appdata\local\programs\python\python35\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101503 entries, 0 to 101502
Data columns (total 10 columns):
RevolvingUtilizationOfUnsecuredLines    101503 non-null float64
age                                     101503 non-null float64
NumberOfTime30-59DaysPastDueNotWorse    101503 non-null float64
MonthlyIncome                           101503 non-null float64
NumberOfOpenCreditLinesAndLoans         101503 non-null float64
NumberOfTimes90DaysLate                 101503 non-null float64
NumberRealEstateLoansOrLines            101503 non-null float64
NumberOfTime60-89DaysPastDueNotWorse    101503 non-null float64
NumberOfDependents                      101503 non-null float64
MonthlyDebts                            101503 non-null float64
dtypes: float64(10)
memory usage: 7.7 MB


In [6]:
y_preds = model.predict_proba(x1)

y_preds =pd.DataFrame(y_preds)
y_preds


,0,1
0,0.970000,0.030000
1,0.970000,0.030000
2,1.000000,0.000000
3,0.990000,0.010000
4,0.820000,0.180000
5,0.990000,0.010000
6,0.840000,0.160000
7,0.940000,0.060000
8,0.890000,0.110000
9,0.516667,0.483333
